In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup

import pandas as pd
from time import sleep
import re
from tqdm.auto import tqdm
import datetime

import os

CHROMEDRIVER = "/opt/ml/crawling/chromedriver_linux_96"

# 드라이브 실행
options = webdriver.ChromeOptions()
# options.add_argument('window-size=1920,1080')
options.add_argument('headless')

#서버 우분투에서 안 하면 오류남
options.add_argument('--no-sandbox')

# Message unknown error: session deleted because of page crash
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(CHROMEDRIVER, options=options)
driver.implicitly_wait(5)


tab_manager = {}

# 크롤링할 링크 수집
home_url = "https://shoppinglive.naver.com/home"
driver.get(url= home_url)
homepage_tab = driver.window_handles[0]

# element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "TitleMoreLinkSection_section_rym8t LiveNow_section_2qq9d")))
sleep(5)


/tmp/ipykernel_39805/942588504.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(CHROMEDRIVER, options=options)


In [3]:
DATA_SAVE_FOLDER_PATH = "/opt/ml/crawling/data/1220_1"
if not os.path.exists(DATA_SAVE_FOLDER_PATH):
    os.makedirs(DATA_SAVE_FOLDER_PATH)

In [4]:
# 특정 태그안의 속성 가져오기
# https://www.jungyin.com/168

def get_page_links(num = 10):
    driver.switch_to.window(homepage_tab)
    driver.get(url= home_url)
    sleep(10)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    #<a rel="opener" target="_self" class="VideoCard_link_3x4f-" href="/lives/330088?fm=shoppinglive&amp;sn=home" style="user-select: auto;">
    live_panels = soup.find_all("a", class_ = "VideoCard_link_3x4f-")
    live_link = [f"https://shoppinglive.naver.com{panel['href']}" for panel in live_panels]
    live_name = [panel.find("span", class_="VideoCard_title_1Sgl6").text for panel in live_panels]
    print(live_name)
    live_link = live_link[:num]

    print("page_num ", len(live_link))
    return live_link

def manage_tab(tab_manager, live_link):
    remove_tab = set(tab_manager.keys()) - set(live_link)
    print(f"remove {remove_tab}")
    for link in remove_tab:
        driver.switch_to.window(tab_manager[link])
        driver.close()
        print(f"remove {link}")
        del tab_manager[link]

    # need_tab
    need_tab = set(live_link) - set(tab_manager.keys()) 
    for link in need_tab:
        try:
            print(f"open {link}")
            driver.switch_to.window(homepage_tab) # 위에서 삭제 후 탭을 변홚시켜줘야한다.
            driver.execute_script(f'window.open("{link}");')
            tab_id = driver.window_handles[-1]
            tab_manager[link] = tab_id
        except Exception as e:                             # 예외가 발생했을 때 실행됨
            print('예외가 발생했습니다.', e)
            print(tab_manager)
            print(link)

    return tab_manager

def remove_slash_space(output_name):
    p = re.compile("[/\s]")
    return p.sub("_", output_name)

def get_commnet_from_html(html):
    soup = BeautifulSoup(html, "html.parser")
    
    program = soup.find("h2", class_="LiveHeader_text_2XGaZ").text
    
    program = remove_slash_space(program)

    num = soup.find_all("span", class_="LiveHeader_count_1GinT")
    play = num[0].text
    love = num[1].text

    comment_panel = soup.find("div", class_ = "Comments_animation_area_23hoF")
    # strong에 고객 id, span에 comment 존재
    # # <div class=""><strong role="presentation" class="Comment_id_3pR4u">카페인충전</strong><span role="presentation" class="Comment_comment_2d0tc"> 오!!!</span></div>
    # master, system의 경우 id가 존재하지 않음
    # # <div class=""><span role="presentation" class="Comment_comment_master_3Aard Comments_green_TJ74_"> 하지만 준비된 재고가 품절시 가격이 원복됩니다 ㅠㅠ</span></div>
    
    # 라이브 종료되면 comment_panel 자체가 없어짐. 'NoneType' object has no attribute 'find_all'

    try: # 라이브 종료되거나 파싱이 제대로 되지 않을 경우 오류 발생. 
        ids = comment_panel.find_all("strong", class_= "Comment_id_3pR4u")
        comments = comment_panel.find_all("span", class_= "Comment_comment_2d0tc")
    except Exception as e:                             # 예외가 발생했을 때 실행됨
        print('예외가 발생했습니다.', e)
        print(program)
        return None, None
        
    id_list = [id.text for id in ids]
    comment_list = [comment.text for comment in comments]
    
    df = pd.DataFrame()
    df["id"] = id_list
    df["comment"] = comment_list
    df["program"] = program

    nowDate = datetime.datetime.now()
    time = nowDate.strftime("%Y-%m-%d_%H%M%S")
    df["time"] = time
    df["love"] = love
    df["play"] = play


    # 댓글이 없는 경우 존재.
    # print(comment_panel)
    # <div class="Comments_animation_area_23hoF" style="transform: translateY(0px); transition-duration: 0.5s;"></div>
    try:
        print(f"프로그램 {program} 시간 {time} 채팅 {comment_list[-1]}")
    except Exception as e:                             # 예외가 발생했을 때 실행됨
        print('예외가 발생했습니다.', e)
        print(comment_panel)

    # 카테고리, 누적시장자수 

    # 기존 파일 덮어쓰기.
    # 재접속하지 않는다는 가정하에
    
    
    df_path = os.path.join(DATA_SAVE_FOLDER_PATH, f"{program}.csv")

    if os.path.exists(df_path):
        prev_df = pd.read_csv(df_path)
        # if prev_df.iloc[0]
        try:
            df.loc[prev_df.index, ["time", "love", "play"]] = prev_df[["time", "love", "play"]]
        except Exception as e:     
            print('df')
            print(df)
            print("prev_df")
            print(prev_df)
    
    # 저장
    df.to_csv(df_path, index =False)

    return df, program

In [5]:
get_page_links(5)

['토마호크&우대갈비&포터하우스 스테이크로 연말 분위기🎉', '연말선물 고민끝! 시리어스와 함께 따뜻한 연말 보내세요', '[하츠] 🎁크리스마스 LIVE 특가🎁 공기청정기', '홀리엘의 쇼핑라이브', '12월 20일 신상 할인 라이브', '한파대비🍰비교불가 프리미어  아우터 대박 할인전💖2', 'UNMe Kids🌹10원 EVENT👏신상 Sale', '삼성파트너 쇼마젠시의 쇼핑라이브', '내일 뭐 입지 10%+5%+추가할인 1부', '12/20) 강남가로수길 유니짜장과함께하는코디반점', '[탑탑데이X리틀마켓] 가성비 좋은 겨울 #데일리룩', '[제이시스터] 겨울신상 라방특가 추가 5%할인 ! ', '겨울철 따뜻한 간식 꿀맛나는 무안 꿀고구마!', '2부 세일세일세일 어서 들어오셔요~~^^', '마리러브의 쇼핑라이브', '아기옷 유아 설빔 한복 슈트 특집', '뽀글이벙거지 귀도리비니 2+1', '머이리 리얼리티해~🖤', '⭐증정이벤트⭐숨쉬기 편한 KF80/94/컬러 마스크', '블루비 순간 난방 욕실 난방기 화장실 히터 3종~']
page_num  5


['https://shoppinglive.naver.com/lives/360022?fm=shoppinglive&sn=home',
 'https://shoppinglive.naver.com/lives/357102?fm=shoppinglive&sn=home',
 'https://shoppinglive.naver.com/lives/356996?fm=shoppinglive&sn=home',
 'https://shoppinglive.naver.com/lives/359947?fm=shoppinglive&sn=home',
 'https://shoppinglive.naver.com/lives/361088?fm=shoppinglive&sn=home']